In [1]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer


In [2]:
pd.set_option('display.max_colwidth',100000)

In [3]:
tweets_df = pd.read_csv('tweets.csv', index_col=0)

Let's include statistics on whether or not a tweet is a retweet- it could be useful in the future.  

In [4]:
tweets_df['retweet'] = tweets_df.full_text.apply(lambda x: int(x.startswith('RT')))

Let's first preprocess our data. The following preprocessor function removes punctuation and emoticons. In addition, it will remove trailing whitespace and lowercase all of the letters. 

In [5]:
def preprocessor(text):
    text = re.sub('@[A-Za-z0-9]+', '', text)
    text = re.sub('https?://[A-Za-z0-9./]+', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()))
    return text + ' '

In [6]:
tweets_df['filtered_tweets'] = tweets_df.full_text.apply(preprocessor)

Let's also include the common English stopwords as well as `rt` since all retweets start with those letters. In addition, I am going to remove the phrase `amp` since these are ampersands and managed to sneak through the original filter. We will then use lemmatization imported from `spacy` in order accurately convey the root words of our vocabulary. 

In [7]:
import spacy
nlp = spacy.load('en')
stop = stopwords.words('english')
stop.append('rt')
stop.append('amp')

In [8]:
def lemmatize(tweet):
    #only return lemmatized version of word if not in stopwords
    return ' '.join([word.lemma_ for word in nlp(tweet) if str(word) not in stop])

In [9]:
tweets_df['lemmatized'] = tweets_df.filtered_tweets.apply(lemmatize)

In [10]:
tweets_df.head()

,full_text,screen_name,retweet,filtered_tweets,lemmatized
0,"We are mobilizing to support those impacted by the #CampFire, #HillFire and #WoolseyFire with SOS Alerts, and committing resources from @googleorg to aid those in need. Our thoughts are with all those displaced by these fires.",sundarpichai,0,we are mobilizing to support those impacted by the campfire hillfire and woolseyfire with sos alerts and committing resources from to aid those in need our thoughts are with all those displaced by these fires,mobilize support impact campfire hillfire woolseyfire sos alert commit resource aid need thought displace fire
1,RT @stshank: The @GoogleAI and @googlecloud folks are helping the @nytimes process 5-7 million analog photos from its morgue dating back to…,sundarpichai,1,rt the and folks are helping the process 5 7 million analog photos from its morgue dating back to,folk help process 5 7 million analog photo morgue date back
2,RT @Google: Go Vote: Here’s where to find the information you need to make your voice heard on #ElectionDay.,sundarpichai,1,rt go vote here s where to find the information you need to make your voice heard on electionday,go vote find information need make voice hear electionday
3,RT @AndrewCrow: Amazing change to Google's site. #GoVote https://t.co/cnQwJp09TI,sundarpichai,1,rt amazing change to google s site govote,amazing change google site govote
4,"We’re announcing our AI for Social Good program, applying @GoogleAI expertise to projects w/ positive societal impact. We invite researchers &amp; orgs to submit ideas for the #GoogleAI Impact Challenge: @googleorg will fund $25M in grants to chosen projects https://t.co/W3BMoueTYo",sundarpichai,0,we re announcing our ai for social good program applying expertise to projects w positive societal impact we invite researchers amp orgs to submit ideas for the googleai impact challenge will fund 25m in grants to chosen projects,announce ai social good program apply expertise project w positive societal impact invite researcher org submit idea googleai impact challenge fund 25 grant choose project


In [12]:
tweets_df.to_csv('cleaned_tweets.csv')

In [13]:
user_tweets = tweets_df.groupby('screen_name').lemmatized.sum()

In [14]:
user_tweets_df = pd.DataFrame(data=user_tweets.values, index=user_tweets.index)

In [15]:
user_tweets_df.columns = ['documents']

In [16]:
user_tweets_df.head(1)

documents
screen_name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

Looking at the output above it looks like there is a delimiter labeled `'-PRON'` between each tweet.  Let's get rid of these. 

In [17]:
user_tweets_df['documents'] = user_tweets_df['documents'].apply(lambda x: str(x).replace('-PRON-', ''))

In [18]:
user_tweets_df.head(1)

documents
screen_name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [19]:
user_tweets_df.to_csv('twitter_users.csv')